In this notebook, we will save the target word in each sentence. Because we want to augment the embedding wegiht of target word to evaluate whether or not the attention is useful. 

In [1]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path

# 1 raw data to soup object 

In [2]:
train_file = Path.cwd().parent.joinpath('raw-data/semeval-2016/train.xml')
train_file

PosixPath('/Users/smap10/Project/aspect-extraction/raw-data/semeval-2016/train.xml')

In [3]:
soup = None
with train_file.open(encoding="utf-8") as f:
    soup = BeautifulSoup(f.read().strip(), "lxml-xml")
if soup is None:
    raise Exception("Can't read xml file")
sentence_nodes = soup.find_all("sentence")

In [4]:
for i, node in enumerate(sentence_nodes):
    print(node)
    print()
    if i > 5:
        break

<sentence id="1004293:0">
<text>Judging from previous posts this used to be a good place, but not any longer.</text>
<Opinions>
<Opinion category="RESTAURANT#GENERAL" from="51" polarity="negative" target="place" to="56"/>
</Opinions>
</sentence>

<sentence id="1004293:1">
<text>We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.</text>
<Opinions>
<Opinion category="SERVICE#GENERAL" from="75" polarity="negative" target="staff" to="80"/>
</Opinions>
</sentence>

<sentence id="1004293:2">
<text>They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.</text>
<Opinions>
<Opinion category="SERVICE#GENERAL" from="0" polarity="negative" target="NULL" to="0"/>
</Opinions>
</sentence>

<sentence id="1004293:3">
<text>The food was lousy - too sweet or too salty and the portions tiny.</text>
<Opinions>
<Opinion category="FOOD#QUALITY" from="4" polari

# 2  convert soup object to a list of dictionaries

In [5]:
def soup2dict(sentence_nodes):
    """
    Input: a soup object, e.g. soup.find_all("sentence")
    Output: a list of dictionaries, contains id, text, aspect terms
    """
    sentences = []
    i = 0
    for n in sentence_nodes:
        i += 1
        sentence = {}
        aspect_term = []
        sentence['id'] = i
        sentence['text'] = n.find('text').string
        if n.find('Opinions'):
            for c in n.find('Opinions').contents:
                if c.name == 'Opinion':
                    if c['target'] not in aspect_term:
                        aspect_term.append(c['target'])

        sentence['aspect'] = aspect_term
        sentences.append(sentence)
        
    return sentences

In [6]:
sentences = soup2dict(sentence_nodes)

In [7]:
sentences[:5]

[{'aspect': ['place'],
  'id': 1,
  'text': 'Judging from previous posts this used to be a good place, but not any longer.'},
 {'aspect': ['staff'],
  'id': 2,
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.'},
 {'aspect': ['NULL'],
  'id': 3,
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.'},
 {'aspect': ['food', 'portions'],
  'id': 4,
  'text': 'The food was lousy - too sweet or too salty and the portions tiny.'},
 {'aspect': ['NULL'],
  'id': 5,
  'text': 'After all that, they complained to me about the small tip.'}]

See how many sentences contain aspect terms

In [8]:
# No sentence contain two NULL target (aspect term)
aspect_zero = 0
aspect_one = 0

for s in sentences:
    # aspect == [] or aspect == ['NULL']
    if len(s['aspect'])==0 or s['aspect'][0] == 'NULL':
        aspect_zero += 1
    else:
        aspect_one += 1
        
print(len(sentences)) # Total 2000 sentences
print(aspect_zero) # xxx sentences have no aspect term
print(aspect_one) # xxx sentences have at least 1 aspect term

2000
791
1209


In [18]:
def xml2soup(path):
    soup = None
    with path.open(encoding="utf-8") as f:
        soup = BeautifulSoup(f.read().strip(), "lxml-xml")
    if soup is None:
        raise Exception("Can't read xml file")
    sentence_nodes = soup.find_all("sentence")

    return sentence_nodes

def soup2dict(sentence_nodes):
    """
    Input: a soup object, e.g. soup.find_all("sentence")
    Output: a list of dictionaries, contains id, text, aspect terms
    """
    sentences = []
    i = 0
    for n in sentence_nodes:
        i += 1
        sentence = {}
        aspect_term = []
        sentence['id'] = i
        sentence['text'] = n.find('text').string
        if n.find('Opinions'):
            for c in n.find('Opinions').contents:
                if c.name == 'Opinion':
                    if c['target'] not in aspect_term:
                        aspect_term.append(c['target'])

        sentence['aspect'] = aspect_term
        sentences.append(sentence)
        
    return sentences

## 3 save target word as a list

In [14]:
def save_target(sentences):
    target_list = []
    for s in sentences:
        # aspect == [] or aspect == ['NUMM']
        target_list.append(s['aspect'])
    return target_list

In [21]:
# Path
train_path = Path.cwd().parent.joinpath('raw-data/semeval-2016/train.xml')
test_path = Path.cwd().parent.joinpath('raw-data/semeval-2016/test.xml')

In [22]:
# soup nodes
train_nodes = xml2soup(train_path)
test_nodes = xml2soup(test_path)

In [23]:
# sentence dictionary
train_sentences = soup2dict(train_nodes)
test_sentences = soup2dict(test_nodes)

In [30]:
# save word in each aspect
train_target = save_target(train_sentences)
test_target = save_target(test_sentences)

In [31]:
print(len(train_target))
print(train_target[:5])
print(len(test_target))
print(test_target[:5])

2000
[['place'], ['staff'], ['NULL'], ['food', 'portions'], ['NULL']]
676
[['NULL'], ['sushi'], ['portions'], ['Green Tea creme brulee'], ['NULL']]


In [32]:
# save to file
train_target = np.array(train_target)
test_target = np.array(test_target)

In [33]:
np.save('train_target.npy', train_target) 
np.save('test_target.npy', test_target) 

In [35]:
ls

anago-embedding.ipynb
aspect-based-dataset-IBO-Readable-with-target.ipynb
aspect-based-dataset-IBO-Readable.ipynb
aspect-based-dataset-IBO.ipynb
bi-lstm-crf-embedding.ipynb
bi-lstm-crf.ipynb
load-large-file-with-progress-bar.ipynb
